In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (142 kB/s)
Reading package lists... Done
Building dependency tree       
Reading s

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion De Datos/Trabajo_practico/contents.csv', header=True, inferSchema=True)
rdd = df.rdd

In [ ]:
#Para los contenidos visibles en wikipedia, cuales son los artículos que tienen la máxima y mínima distancia entre ids de su revisión actual y la anterior (⭐⭐)

In [ ]:
rdd_filrado = rdd.filter(lambda x:x.revision_id != None and x.parent_revision!=None).cache()  # Filtro de None´s

In [ ]:
rdd_filtrado = rdd_filrado.filter(lambda x: x.revision_id.isnumeric() and x.parent_revision.isnumeric()).cache() # Filtro de aquellos que den false en isnumeric

In [ ]:
rdd_filtrado.map(lambda x: (x.id, (int(x.revision_id) - int(x.parent_revision))) ).take(5)  # Para ver

[('5', 16676), ('7', 7582), ('10', 141661), ('15', 4320405), ('17', 1456366)]

In [ ]:
rdd_limpio = rdd_filtrado.map(lambda x: (x.id, (int(x.revision_id) - int(x.parent_revision))) ).filter(lambda x:x[1]>=0)  # Filtro aquellos con distancia negativa

Muestro con un takeOrdered por si hay más de una con la maxima (en este caso no)

In [ ]:
rdd_limpio.takeOrdered(5, key=lambda x:-x[1])

[('30623', 137146746),
 ('21749', 136997871),
 ('15167', 136797279),
 ('165391', 136714513),
 ('42465', 136703240)]

Muestro la máxima

In [ ]:
rdd_limpio.reduce(lambda a,b:(a and b if a[1]==b[1] else a if a[1]>b[1] else b))

('30623', 137146746)

Muestro con el takeOrdered por si hay más de una con la minima (acá si)

In [ ]:
rdd_limpio.takeOrdered(5, key=lambda x:x[1])

[('19', 1), ('32', 1), ('51', 1), ('295', 1), ('299', 1)]

Muestro la mínima

In [ ]:
rdd_limpio.reduce(lambda a,b:(a if a[1]<b[1] else (a and b if a[1]==b[1] else b)))

('10010546', 1)